In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


In [2]:
# Config

BATCH_SIZE = 32
IMG_SIZE = 224
EPOCHS_PHASE_A = 10   
EPOCHS_PHASE_B = 5   
LR_PHASE_A = 1e-3
LR_PHASE_B = 1e-4
DATA_ROOT = r"D:\dataset_split"
N_LAST_LAYERS = 20   

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [3]:
# Data

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness = 0.2, contrast = 0.2, saturation = 0.2),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(root = f"{DATA_ROOT}/train", transform = train_transform)
val_dataset   = datasets.ImageFolder(root = f"{DATA_ROOT}/val", transform = val_transform)
test_dataset  = datasets.ImageFolder(root = f"{DATA_ROOT}/test", transform = val_transform)

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_loader   = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False)
test_loader  = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False)

num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)


Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [4]:
# Model: MobileNetV2

mobilenet = models.mobilenet_v2(pretrained=True)

# Replace classifier
mobilenet.classifier[1] = nn.Linear(mobilenet.classifier[1].in_features, num_classes)
model = mobilenet.to(device)

criterion = nn.CrossEntropyLoss()


# Training function

def train_model(model, criterion, optimizer, train_loader, val_loader, epochs, phase_name = "Phase"):
    for epoch in range(epochs):
        # Train
        model.train()
        running_loss, running_correct = 0.0, 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            running_correct += (outputs.argmax(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_correct / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * imgs.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)

        print(f"{phase_name} Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
              f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    return model


# Phase A: Train classifier

for param in model.parameters():
    param.requires_grad = False
for param in model.classifier[1].parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.classifier[1].parameters(), lr = LR_PHASE_A)
model = train_model(model, criterion, optimizer, train_loader, val_loader,
                    epochs = EPOCHS_PHASE_A, phase_name = "Phase A")


# Phase B: Fine-tuning last N layers

# Freeze all first
for name, param in model.named_parameters():
    param.requires_grad = False

# Unfreeze last N layers + classifier
for name, param in list(model.named_parameters())[-N_LAST_LAYERS:]:
    param.requires_grad = True

# Optimizer (only train unfrozen layers)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = LR_PHASE_B)

# Train Phase B
model = train_model(model, criterion, optimizer, train_loader, val_loader,
                    epochs = EPOCHS_PHASE_B, phase_name = "Phase B")

d:\Anaconda\envs\plant\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\plant\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Phase A Epoch [1/10] Train Loss: 0.7052 | Train Acc: 0.8058 | Val Loss: 0.3265 | Val Acc: 0.9086
Phase A Epoch [2/10] Train Loss: 0.3735 | Train Acc: 0.8831 | Val Loss: 0.2398 | Val Acc: 0.9295
Phase A Epoch [3/10] Train Loss: 0.3197 | Train Acc: 0.8980 | Val Loss: 0.1931 | Val Acc: 0.9402
Phase A Epoch [4/10] Train Loss: 0.2986 | Train Acc: 0.8996 | Val Loss: 0.1888 | Val Acc: 0.9436
Phase A Epoch [5/10] Train Loss: 0.2850 | Train Acc: 0.9041 | Val Loss: 0.1953 | Val Acc: 0.9359
Phase A Epoch [6/10] Train Loss: 0.2850 | Train Acc: 0.9054 | Val Loss: 0.1814 | Val Acc: 0.9402
Phase A Epoch [7/10] Train Loss: 0.2678 | Train Acc: 0.9116 | Val Loss: 0.1745 | Val Acc: 0.9397
Phase A Epoch [8/10] Train Loss: 0.2619 | Train Acc: 0.9106 | Val Loss: 0.1921 | Val Acc: 0.9320
Phase A Epoch [9/10] Train Loss: 0.2571 | Train Acc: 0.9138 | Val Loss: 0.1939 | Val Acc: 0.9363
Phase A Epoch [10/10] Train Loss: 0.2564 | Train Acc: 0.9127 | Val Loss: 0.1397 | Val Acc: 0.9563
Phase B Epoch [1/5] Train Los

In [5]:
# Test

model.eval()
test_loss, test_correct = 0.0, 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * imgs.size(0)
        test_correct += (outputs.argmax(1) == labels).sum().item()

test_loss /= len(test_loader.dataset)
test_acc = test_correct / len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")


# Save

torch.save(model.state_dict(), "D:/saved_models/mobilenetv2_finetuned.pt")


Test Loss: 0.0730 | Test Acc: 0.9735
